# River Time Series Extender (Simplified with xfvcom)
**Author: Jun Sasaki | Created: 2025-01-04**

**Purpose:** Simplified version using xfvcom functionality and proposed extensions

This notebook demonstrates:
1. How to leverage existing xfvcom components
2. Proposed utility functions to reduce code duplication
3. A cleaner approach to river time series extension

## 1. Setup and Imports

In [ ]:
from pathlib import Path
from datetime import datetime
import numpy as np
import pandas as pd
import netCDF4 as nc
import matplotlib.pyplot as plt

# Import xfvcom components
from xfvcom.io.sources.utils import load_timeseries_table

# Note: These functions would be added to xfvcom
# For now, we'll define them locally
print("Setup complete")

## 2. Proposed xfvcom Extensions

These functions should be added to `xfvcom/utils/time_utils.py` and `xfvcom/io/netcdf_utils.py`:

In [ ]:
# These should be in xfvcom/utils/time_utils.py

def decode_fvcom_time(itime: np.ndarray, itime2: np.ndarray) -> pd.DatetimeIndex:
    """Decode FVCOM time format (MJD) to pandas datetime."""
    mjd_epoch = pd.Timestamp('1858-11-17')
    times = []
    for day, ms in zip(itime, itime2):
        dt = mjd_epoch + pd.Timedelta(days=int(day)) + pd.Timedelta(milliseconds=int(ms))
        times.append(dt)
    return pd.DatetimeIndex(times)


def encode_fvcom_time(datetimes: pd.DatetimeIndex) -> tuple[np.ndarray, np.ndarray, list[str]]:
    """Encode datetime to FVCOM time format (MJD)."""
    mjd_epoch = pd.Timestamp('1858-11-17')
    
    itime = np.zeros(len(datetimes), dtype=np.int32)
    itime2 = np.zeros(len(datetimes), dtype=np.int32)
    times_str = []
    
    for i, dt in enumerate(datetimes):
        delta = dt - mjd_epoch
        itime[i] = delta.days
        
        midnight = dt.replace(hour=0, minute=0, second=0, microsecond=0)
        itime2[i] = int((dt - midnight).total_seconds() * 1000)
        
        times_str.append(dt.strftime('%Y-%m-%d %H:%M:%S.%f')[:-3])
    
    return itime, itime2, times_str


def to_mjd(times: pd.DatetimeIndex) -> np.ndarray:
    """Convert to Modified Julian Day (float format for comparison with existing code)."""
    mjd_epoch = pd.Timestamp('1858-11-17')
    return ((times - mjd_epoch) / pd.Timedelta('1D')).to_numpy('f8')

In [ ]:
# These should be in xfvcom/io/netcdf_utils.py

def read_fvcom_river_nc(filepath: Path) -> dict:
    """Read FVCOM river NetCDF with proper decoding."""
    data = {}
    
    with nc.Dataset(filepath, 'r') as ds:
        # Store metadata
        data['global_attrs'] = {attr: ds.getncattr(attr) for attr in ds.ncattrs()}
        data['dimensions'] = {dim: len(ds.dimensions[dim]) for dim in ds.dimensions}
        
        # Decode time
        data['datetime'] = decode_fvcom_time(ds.variables['Itime'][:], 
                                            ds.variables['Itime2'][:])
        
        # Determine river dimension name
        river_dim = 'river' if 'river' in ds.dimensions else 'rivers'
        data['river_dim'] = river_dim
        
        # Read river data as pandas DataFrame for easier manipulation
        for var in ['river_flux', 'river_temp', 'river_salt']:
            if var in ds.variables:
                data[var] = pd.DataFrame(ds.variables[var][:], 
                                        index=data['datetime'])
                data[f'{var}_attrs'] = {attr: ds.variables[var].getncattr(attr) 
                                       for attr in ds.variables[var].ncattrs()}
        
        # River names
        if 'river_names' in ds.variables:
            names_raw = ds.variables['river_names'][:]
            data['river_names'] = []
            for i in range(data['dimensions'][river_dim]):
                name = names_raw[i] if names_raw.ndim == 1 else names_raw[i, :]
                if hasattr(name, 'tobytes'):
                    name_str = name.tobytes().decode('utf-8').strip('\x00').strip()
                else:
                    name_str = ''.join([chr(c) if isinstance(c, int) else str(c) 
                                       for c in name.flatten()]).strip('\x00').strip()
                data['river_names'].append(name_str)
    
    return data


def write_fvcom_river_nc(filepath: Path, data: dict) -> None:
    """Write FVCOM-compatible river NetCDF."""
    river_dim = data['river_dim']
    
    with nc.Dataset(filepath, 'w', format='NETCDF4_CLASSIC') as ds:
        # Create dimensions
        n_times = len(data['datetime'])
        ds.createDimension('time', n_times)
        ds.createDimension(river_dim, data['dimensions'][river_dim])
        
        # String dimensions
        if 'river_names' in data:
            max_name_len = max(len(name) for name in data['river_names']) + 1
            ds.createDimension('namelen', max_name_len)
        ds.createDimension('DateStrLen', 30)
        
        # Global attributes
        for attr, value in data.get('global_attrs', {}).items():
            ds.setncattr(attr, value)
        ds.setncattr('history', f"Extended on {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        
        # Time variables
        itime, itime2, times_str = encode_fvcom_time(data['datetime'])
        
        itime_var = ds.createVariable('Itime', 'i4', ('time',))
        itime_var.units = 'days since 1858-11-17 00:00:00'
        itime_var[:] = itime
        
        itime2_var = ds.createVariable('Itime2', 'i4', ('time',))
        itime2_var.units = 'msec since 00:00:00'
        itime2_var[:] = itime2
        
        times_var = ds.createVariable('Times', 'c', ('time', 'DateStrLen'))
        for i, ts in enumerate(times_str):
            ts_array = np.zeros(30, dtype='S1')
            ts_bytes = ts.encode('utf-8')
            ts_array[:len(ts_bytes)] = list(ts_bytes)
            times_var[i, :] = ts_array
        
        # River names
        if 'river_names' in data:
            names_var = ds.createVariable('river_names', 'c', (river_dim, 'namelen'))
            for i, name in enumerate(data['river_names']):
                name_array = np.zeros(max_name_len, dtype='S1')
                name_bytes = name.encode('utf-8')
                name_array[:len(name_bytes)] = list(name_bytes)
                names_var[i, :] = name_array
        
        # River data variables
        for var_name in ['river_flux', 'river_temp', 'river_salt']:
            if var_name in data:
                var = ds.createVariable(var_name, 'f4', ('time', river_dim))
                for attr, value in data.get(f'{var_name}_attrs', {}).items():
                    var.setncattr(attr, value)
                var[:] = data[var_name].values

In [ ]:
# These should be in xfvcom/utils/timeseries_utils.py

def extend_timeseries_ffill(df: pd.DataFrame, 
                           extend_to: str | pd.Timestamp,
                           freq: str = None) -> pd.DataFrame:
    """
    Extend time series using forward fill method.
    
    Parameters
    ----------
    df : pd.DataFrame
        DataFrame with DatetimeIndex
    extend_to : str or pd.Timestamp
        Target end datetime
    freq : str, optional
        Frequency string. If None, inferred from data
    
    Returns
    -------
    pd.DataFrame
        Extended DataFrame
    """
    extend_to = pd.Timestamp(extend_to)
    
    if extend_to <= df.index[-1]:
        return df
    
    # Infer frequency if not provided
    if freq is None:
        freq = pd.infer_freq(df.index)
        if freq is None and len(df.index) > 1:
            # Calculate from first two timestamps
            delta = df.index[1] - df.index[0]
            hours = delta.total_seconds() / 3600
            freq = f'{int(hours)}h' if hours == int(hours) else f'{int(hours * 60)}min'
    
    # Create extended index
    extended_index = pd.date_range(start=df.index[0], end=extend_to, freq=freq)
    
    # Reindex and forward fill
    return df.reindex(extended_index, method='ffill')


def extend_river_nc_file(input_path: Path, 
                        output_path: Path,
                        extend_to: str,
                        method: str = 'ffill') -> None:
    """
    High-level function to extend river NetCDF file.
    
    Parameters
    ----------
    input_path : Path
        Input river NetCDF file
    output_path : Path
        Output river NetCDF file
    extend_to : str
        Target end datetime (YYYY-MM-DD HH:MM:SS)
    method : str
        Extension method ('ffill', 'linear', 'seasonal')
    """
    # Read input file
    data = read_fvcom_river_nc(input_path)
    
    # Extend each variable
    for var in ['river_flux', 'river_temp', 'river_salt']:
        if var in data:
            if method == 'ffill':
                data[var] = extend_timeseries_ffill(data[var], extend_to)
            elif method == 'linear':
                # Could add linear extrapolation
                pass
            elif method == 'seasonal':
                # Could add seasonal pattern repetition
                pass
    
    # Update datetime index
    data['datetime'] = data['river_flux'].index if 'river_flux' in data else data['river_temp'].index
    
    # Write output file
    write_fvcom_river_nc(output_path, data)
    
    print(f"Extended {input_path.name} to {extend_to}")
    print(f"Output saved to {output_path}")

## 3. Simple Usage Example

In [ ]:
# Define paths
base_path = Path("~/Github/TB-FVCOM/goto2023").expanduser()
input_file = base_path / "input/2020" / "TokyoBay2020kisarazufinal_sewer.nc"
output_dir = Path("extended_river_files")
output_dir.mkdir(exist_ok=True)
output_file = output_dir / "river_extended_simple.nc"

# Extend the file with one function call
extend_river_nc_file(
    input_path=input_file,
    output_path=output_file,
    extend_to="2021-12-31 23:00:00",
    method='ffill'
)

## 4. Verify and Visualize

In [ ]:
# Read both files for comparison
original = read_fvcom_river_nc(input_file)
extended = read_fvcom_river_nc(output_file)

# Quick visualization
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

for ax, var, title in zip(axes, 
                          ['river_flux', 'river_temp', 'river_salt'],
                          ['Discharge', 'Temperature', 'Salinity']):
    if var in original:
        # Plot first river only for simplicity
        ax.plot(original[var].iloc[:, 0], label='Original', linewidth=2)
        ax.plot(extended[var].iloc[:, 0], '--', label='Extended', alpha=0.7)
        ax.axvline(x=original['datetime'][-1], color='red', linestyle=':', alpha=0.5)
        ax.set_title(title)
        ax.set_xlabel('Time')
        ax.legend()
        ax.grid(True, alpha=0.3)

plt.suptitle(f"River: {original['river_names'][0] if 'river_names' in original else 'River 1'}")
plt.tight_layout()
plt.show()

print(f"\nOriginal: {original['datetime'][0]} to {original['datetime'][-1]}")
print(f"Extended: {extended['datetime'][0]} to {extended['datetime'][-1]}")
print(f"Added {len(extended['datetime']) - len(original['datetime'])} time steps")

## 5. Integration with xfvcom River Generator

Here's how this could integrate with the existing `RiverNcGenerator`:

In [ ]:
# Proposed ExtendedTimeSeriesSource for xfvcom/io/sources/extended.py

from xfvcom.io.sources.base import BaseForcingSource

class ExtendedTimeSeriesSource(BaseForcingSource):
    """
    Time series source with automatic extension capabilities.
    
    This would be a new source type that extends existing time series
    data when the requested time range exceeds the available data.
    """
    
    def __init__(self, 
                 path: Path,
                 river_name: str = None,
                 extend_method: str = 'ffill',
                 input_tz: str = "Asia/Tokyo"):
        self.path = path
        self.river_name = river_name
        self.extend_method = extend_method
        self.input_tz = input_tz
        
        # Load data using existing xfvcom utility
        self.data = load_timeseries_table(path)
        
    def get_series(self, var_name: str, out_times: np.ndarray) -> np.ndarray:
        """
        Get time series, automatically extending if needed.
        """
        # Convert out_times from MJD to datetime
        mjd_epoch = pd.Timestamp('1858-11-17')
        out_datetime = pd.to_datetime(out_times * 86400, unit='s', origin=mjd_epoch)
        
        # Get column name
        col = self.river_name if self.river_name else var_name
        
        if col not in self.data.columns:
            raise KeyError(f"Column {col} not found in {self.path}")
        
        # Check if extension is needed
        if out_datetime[-1] > self.data.index[-1]:
            # Extend the data
            if self.extend_method == 'ffill':
                extended = extend_timeseries_ffill(
                    self.data[[col]], 
                    out_datetime[-1]
                )
                return extended[col].reindex(out_datetime).values
        
        # No extension needed, use regular interpolation
        return self.data[col].reindex(out_datetime, method='nearest').values

# Example usage in YAML configuration:
yaml_config = """
# This would go in a YAML config file for RiverNcGenerator
time_series:
  Shibaura: "shibaura_data.csv:flux,temp,salt"
  
# New: Extended time series with automatic forward fill
extended_time_series:
  Shibaura: 
    file: "shibaura_data.csv"
    method: "ffill"  # or "linear", "seasonal"
    extend_to: "2025-12-31"
"""

print("ExtendedTimeSeriesSource class defined")
print("This would allow automatic extension within the existing generator framework")

## Summary of Improvements

### 1. **Code Reduction**
- From ~400 lines to ~150 lines of core functionality
- Reusable functions that can be added to xfvcom

### 2. **Better Integration**
- Uses `load_timeseries_table()` from xfvcom
- Compatible with existing `BaseForcingSource` infrastructure
- Works with YAML configuration system

### 3. **Proposed xfvcom Extensions**

#### A. `xfvcom/utils/time_utils.py`:
- `decode_fvcom_time()`: Decode MJD format
- `encode_fvcom_time()`: Encode to MJD format
- `to_mjd()`: Centralized MJD conversion

#### B. `xfvcom/utils/timeseries_utils.py` (new):
- `extend_timeseries_ffill()`: Forward fill extension
- `extend_timeseries_linear()`: Linear extrapolation
- `extend_timeseries_seasonal()`: Seasonal pattern repetition

#### C. `xfvcom/io/netcdf_utils.py` (new):
- `read_fvcom_river_nc()`: Read river NetCDF files
- `write_fvcom_river_nc()`: Write river NetCDF files
- `extend_river_nc_file()`: High-level extension function

#### D. `xfvcom/io/sources/extended.py` (new):
- `ExtendedTimeSeriesSource`: Auto-extending data source

### 4. **Benefits**
- Consistent with xfvcom's design philosophy
- Reusable across different forcing types (river, met, groundwater)
- Maintains FVCOM format compatibility
- Integrates with existing YAML configuration system